# Global Sustainable Energy Trends Analysis

**Author:** Divya Srinivasa  

**Tools:** Python (pandas, matplotlib, seaborn), SQLite (SQLAlchemy), Tableau  

**Data:** `sustainable energy data.xlsx` (uploaded) — country-year sustainable energy indicators (2000–2024)

---

This notebook prepares the dataset, builds a SQLite database, runs SQL queries for analysis, creates Python visualizations, and exports clean CSVs for Tableau.

## 1. Install & Import Libraries
Run these cells in Colab or a Jupyter environment. If using Colab, enable the notebook to access the uploaded file in `/mnt/data`.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
from sqlalchemy import create_engine
import os

sns.set(style='whitegrid')
print('Libraries imported. Files in /mnt/data:')
print(os.listdir('/mnt/data'))

## 2. Load the Excel file and inspect

In [ ]:
file_path = '/mnt/data/sustainable energy data.xlsx'

# Read Excel
try:
    df = pd.read_excel(file_path)
    print('Shape:', df.shape)
    display(df.head())
    display(pd.Series(df.columns))
except Exception as e:
    print('Error reading file:', e)

## 3. Clean & Preprocess
- Standardize column names
- Convert Year to int
- Handle missing values where appropriate
- Create derived columns (example: net production if available)


In [ ]:
df.columns = df.columns.str.strip().str.replace('[^0-9a-zA-Z_ ]', '', regex=True).str.replace('\n',' ').str.replace('  ',' ').str.strip().str.lower().str.replace(' ', '_')
print('Cleaned columns:')
print(df.columns.tolist())

# Convert year to int if possible
try:
    df['year'] = df['year'].astype(int)
except Exception as e:
    print('Could not convert year to int:', e)

# Coerce numeric columns heuristically
numeric_cols = [c for c in df.columns if any(k in c for k in ['electricity','production','consumption','co2','gdp','per_capita','intensity','capacity','emissions','density','population'])]
for c in numeric_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# Summary of missing values
missing = df.isnull().mean().sort_values(ascending=False)
display(missing.head(15))

# Example derived column
if 'production' in df.columns and 'consumption' in df.columns:
    df['net_production'] = df['production'] - df['consumption']
else:
    df['net_production'] = np.nan

print('Data types:')
display(df.dtypes.head(20))

# Save a cleaned CSV for quick reuse
clean_csv = '/mnt/data/cleaned_sustainable_energy.csv'
df.to_csv(clean_csv, index=False)
print('Cleaned CSV saved to', clean_csv)

## 4. Create SQLite database and load cleaned data

In [ ]:
db_path = '/mnt/data/sustainable_energy.db'
engine = create_engine(f'sqlite:///{db_path}')
df.to_sql('energy_data', engine, if_exists='replace', index=False)
print('Database created at', db_path)
with engine.connect() as conn:
    result = conn.execute('SELECT COUNT(*) FROM energy_data').fetchall()
    print('Total rows in energy_data table:', result[0][0])

## 5. Example SQL Queries and Analysis

In [ ]:
# Top 10 countries by average renewable energy share (safe pandas approach)
if 'renewable_energy_share_in_the_total_final_energy_consumption_' in df.columns:
    tmp = df[['entity','year','renewable_energy_share_in_the_total_final_energy_consumption_']].copy()
    tmp['renewable_share'] = pd.to_numeric(tmp['renewable_energy_share_in_the_total_final_energy_consumption_'], errors='coerce')
    top10 = tmp.groupby('entity')['renewable_share'].mean().sort_values(ascending=False).head(10).reset_index().rename(columns={'renewable_share':'avg_share'})
    display(top10)
else:
    print('Column renewable_energy_share_in_the_total_final_energy_consumption_ not found. Available columns:')
    print(df.columns.tolist())

In [ ]:
# Yearly averages for CO2 emissions and renewable share
cols = []
if 'value_co2_emissions_kt_by_country' in df.columns:
    cols.append('value_co2_emissions_kt_by_country')
if 'renewable_energy_share_in_the_total_final_energy_consumption_' in df.columns:
    cols.append('renewable_energy_share_in_the_total_final_energy_consumption_')

if cols:
    tmp = df[['year'] + cols].copy()
    for c in cols:
        tmp[c] = pd.to_numeric(tmp[c], errors='coerce')
    yearly = tmp.groupby('year')[cols].mean().reset_index()
    display(yearly.head())
    plt.figure(figsize=(10,5))
    if 'renewable_energy_share_in_the_total_final_energy_consumption_' in yearly.columns:
        plt.plot(yearly['year'], yearly['renewable_energy_share_in_the_total_final_energy_consumption_'], marker='o', label='Avg Renewable Share (%)')
    if 'value_co2_emissions_kt_by_country' in yearly.columns:
        plt.plot(yearly['year'], yearly['value_co2_emissions_kt_by_country'], marker='s', label='Avg CO2 Emissions (kt)')
    plt.xlabel('Year')
    plt.legend()
    plt.title('Yearly Avg: Renewable Share vs CO2 Emissions')
    plt.show()
else:
    print('Required columns for trend not found.')

### Electricity access growth: Top countries

In [ ]:
if 'access_to_electricity__of_population_' in df.columns:
    tmp = df[['entity','year','access_to_electricity__of_population_']].copy()
    tmp['access_to_elec'] = pd.to_numeric(tmp['access_to_electricity__of_population_'], errors='coerce')
    growth = tmp.groupby('entity').agg(min_year=('year','min'), max_year=('year','max'), min_val=('access_to_elec','min'), max_val=('access_to_elec','max')).reset_index()
    growth['growth_pct'] = growth['max_val'] - growth['min_val']
    top_growth = growth.sort_values('growth_pct', ascending=False).head(10)
    display(top_growth[['entity','min_year','max_year','min_val','max_val','growth_pct']])
else:
    print('access_to_electricity__of_population_ column not found.')

## 6. Python Visualizations
- Line charts, scatter plots, and maps can be created. Below: scatter of GDP per capita vs renewable share.

In [ ]:
if 'gdp_per_capita' in df.columns and 'renewable_energy_share_in_the_total_final_energy_consumption_' in df.columns:
    plot_df = df[['entity','year','gdp_per_capita','renewable_energy_share_in_the_total_final_energy_consumption_']].copy()
    plot_df['renewable_share'] = pd.to_numeric(plot_df['renewable_energy_share_in_the_total_final_energy_consumption_'], errors='coerce')
    plot_df['gdp_per_capita'] = pd.to_numeric(plot_df['gdp_per_capita'], errors='coerce')
    sample = plot_df.dropna(subset=['renewable_share','gdp_per_capita']).sample(frac=0.2, random_state=1)
    plt.figure(figsize=(8,6))
    sns.scatterplot(data=sample, x='gdp_per_capita', y='renewable_share', alpha=0.7)
    plt.xlabel('GDP per Capita')
    plt.ylabel('Renewable Energy Share (%)')
    plt.title('GDP per Capita vs Renewable Energy Share (sample)')
    plt.show()
else:
    print('Required columns for scatter not found.')

## 7. Export cleaned summary for Tableau
Export a summarized CSV (Entity, Year, key metrics) for Tableau import.

In [ ]:
export_cols = ['entity','year']
candidates = ['access_to_electricity__of_population_','renewable_energy_share_in_the_total_final_energy_consumption_','value_co2_emissions_kt_by_country','gdp_per_capita','density']
existing = [c for c in df.columns if c in candidates]
export_cols += existing
summary = df[export_cols].copy()
summary.to_csv('/mnt/data/tableau_energy_summary.csv', index=False)
print('Exported summary to /mnt/data/tableau_energy_summary.csv')

## 8. Next steps and suggestions

- Use Tableau to build interactive dashboards (maps, time-series, scatter plots).  
- Consider normalizing CO2 emissions by population or GDP for fairness.  
- Add country-level shapefile or GeoJSON for mapping in Tableau.  

---

**Deliverables in `/mnt/data`:**  
- `cleaned_sustainable_energy.csv` (cleaned data)  
- `sustainable_energy.db` (SQLite DB)  
- `tableau_energy_summary.csv` (Tableau-ready summary)  

Good luck! Add interpretation text and policy implications to make this portfolio-ready.